In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision.datasets import MNIST
import torch.nn.functional as F
import wandb

In [12]:
wandb.init(project="mnist-model-evaluation", entity="kimkj")

In [13]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)  # input channel, output channels, kernel size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [18]:
wandb.watch(net, log="all")


[]

In [19]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # log every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            wandb.log({"loss": running_loss / 100})
            running_loss = 0.0

[1, 100] loss: 0.5781414509564639
[1, 200] loss: 0.15523146169260144
[1, 300] loss: 0.10032842166256159
[1, 400] loss: 0.09682094195857643
[1, 500] loss: 0.06656494897790254
[1, 600] loss: 0.0676163771096617
[1, 700] loss: 0.07351508445106447
[1, 800] loss: 0.06918032933492213
[1, 900] loss: 0.05287628105375916
[2, 100] loss: 0.042560564717277885
[2, 200] loss: 0.04525746748899109
[2, 300] loss: 0.042832578056259084
[2, 400] loss: 0.04246213944396004
[2, 500] loss: 0.038827738310210407
[2, 600] loss: 0.04034471719700378
[2, 700] loss: 0.04610825837822631
[2, 800] loss: 0.04225393576081842
[2, 900] loss: 0.04430133920279331
[3, 100] loss: 0.02752765033626929
[3, 200] loss: 0.031773455184302295
[3, 300] loss: 0.031107265628525057
[3, 400] loss: 0.024354798923013732
[3, 500] loss: 0.032761732968501746
[3, 600] loss: 0.026737645419925685
[3, 700] loss: 0.027762318205204792
[3, 800] loss: 0.03084920658002375
[3, 900] loss: 0.036144664086168635
[4, 100] loss: 0.02086902049544733
[4, 200] los

In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
wandb.log({"accuracy": 100 * correct / total})

Accuracy of the network on the 10000 test images: 10 %


In [20]:
wandb.finish()


accuracy,▁
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,10.23
loss,0.01074
